### Creating Sample File

In [40]:
import xml.etree.cElementTree as ET
#Reference: this code was modified from 'Wrangle OpenStreetMap Data' 
#2nd section: 'Project Details'.

OSM_FILE = 'bali.osm'  
SAMPLE_FILE = 'sample.osm'

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    '''Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-
    xml-file-generated-via-xml-etree-elementtree-in-python
    '''
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### Exploring the Structure of the OSM File

In [1]:
'''
1. Use the iterative parsing to process the map file. 
2. Identify what tags and subtags are in the file, as well as how many they are. 
3. Identify the attributes of child and lower level elements
4. Return a dictionary with the tag name as the key 
   and the number of times this tag is encountered in the map as the value.
'''
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import pprint
import cerberus
import schema
from collections import Counter,defaultdict
OSMFILE = 'bali.osm'

def count_tags(elem):
    '''
    For each input element, return a dict with the name of tags
    in the lower level elements as keys and frequency as values.
    '''
    tag_dict = {}
    for sub_elem in elem:
        tag_dict[sub_elem.tag] = tag_dict.get(sub_elem.tag,0) + 1
    return tag_dict

def elem_attrib(elem):
    '''
    For each input element, return a dict with the name of tags 
    in the lower level elements as keys and attributes as values.
    '''
    attrib_dict = {}
    for sub_elem in elem:
        if sub_elem.tag not in attrib_dict:
            attrib_dict[sub_elem.tag] = sub_elem.attrib.keys()
        else:
            for attrib in sub_elem.attrib:
                if attrib not in attrib_dict[sub_elem.tag]:
                    attrib_dict[sub_elem.tag].append(attrib)
    return attrib_dict

def merge_dict(dict1,dict2):
    '''
    Reference: this code was copied and modified from 
    https://stackoverflow.com/questions/1495510/
    combining-dictionaries-of-lists-in-python
    '''
    keys = set(dict1).union(dict2)
    no = []
    return dict((key,list(set(dict1.get(key,no)+dict2.get(key,no)))) for key in keys)
    
def explore():
    tree = ET.parse(OSMFILE)
    root = tree.getroot()
    child_tags = count_tags(root)
    
    print('Tags and their frequency in the child elements:')
    pprint.pprint(child_tags)
    
    print('\nTags and attributes of the child elements:')
    child_attrib = elem_attrib(root)
    pprint.pprint(child_attrib)
    
    sub_tags, tag_attrib, sub_tags_attrib = {}, {}, {}
    for key in child_tags.keys():
        sub_tags[key] = Counter({})
        sub_tags_attrib[key] = {}
        
    for child in root:
        if child.iter():
            sub_tags[child.tag] += Counter(count_tags(child))
            sub_tags_attrib[child.tag] = merge_dict(sub_tags_attrib[child.tag],elem_attrib(child))
        
    print('\nDictionary of tags and their frequency in the lower level elements:')    
    pprint.pprint(sub_tags)
    
    print('\nDictionary of tags and their attributes in the lower level elements:')    
    pprint.pprint(sub_tags_attrib)
    
if __name__ == '__main__':
    explore()

Tags and their frequency in the child elements:
{'bounds': 1,
 'meta': 1,
 'node': 489536,
 'note': 1,
 'relation': 196,
 'way': 79414}

Tags and attributes of the child elements:
{'bounds': ['minlat', 'maxlon', 'minlon', 'maxlat'],
 'meta': ['osm_base'],
 'node': ['changeset',
          'uid',
          'timestamp',
          'lon',
          'version',
          'user',
          'lat',
          'id'],
 'note': [],
 'relation': ['changeset', 'uid', 'timestamp', 'version', 'user', 'id'],
 'way': ['changeset', 'uid', 'timestamp', 'version', 'user', 'id']}

Dictionary of tags and their frequency in the lower level elements:
{'bounds': Counter(),
 'meta': Counter(),
 'node': Counter({'tag': 39038}),
 'note': Counter(),
 'relation': Counter({'member': 1300, 'tag': 1031}),
 'way': Counter({'nd': 587609, 'tag': 111931})}

Dictionary of tags and their attributes in the lower level elements:
{'bounds': {},
 'meta': {},
 'node': {'tag': ['k', 'v']},
 'note': {},
 'relation': {'member': ['ref'

### Auditing and Improving Street Names

In [1]:
'''
This code fixes the street names in OSM file.
The function takes a string with street name as an argument and returns the fixed name
There are four major types of errors:
1. Inconsistency: abbreviation, capitalization, punctuation
2. Missing the word 'Jalan', Indonesian of 'Street'.
3. Inclusion of city and province names
4. Using English word, 'Road'
Reference: this code was modified from 'Case study: OpenStreetMap Data[SQL] Quiz 10'
'''
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import pprint
import cerberus
import schema
from collections import Counter,defaultdict
OSMFILE = 'bali.osm'

mapping_street = {  'Jl.': 'Jalan',
                    'Jl': 'Jalan',
                    'Jln': 'Jalan',
                    'Jln.': 'Jalan',
                    'Jl,': 'Jalan',
                    'JL.': 'Jalan',
                    'JLN.': 'Jalan',
                    'jalan': 'Jalan',
                    'JL': 'Jalan',
                    'JALAN': 'Jalan',
                    'Jalan.': 'Jalan'}

def is_street_name(elem):
    '''
    #Check if the element contains a street name.
    '''
    return (elem.attrib['k'] == 'addr:street')

def audit_street_type(street_types, street_name):
    '''
    Classify the types of errors and store in a dictionary.
    '''
    if street_name.split(' ')[0] != 'Jalan':
        if street_name.split(' ')[0] in mapping_street.keys():
            street_types[street_name.split(' ')[0]].add(street_name)
        elif len(street_name.split(',')) > 1 and street_name.split(',')[1][:4] == ' JL.':
            street_types['JL._invert'].add(street_name)
        elif street_name.split(' ')[0][:3] == 'Jl.':
            street_types['Jl_no_space'].add(street_name)
        elif len(street_name.split(',')) > 1:
            if street_name.split(',')[0][:5] == 'Jalan':
                street_types['extra'].add(street_name)
            else:
                street_types['extra_no_Jalan'].add(street_name)
        elif street_name[-4:] == 'Road':
            street_types['Road'].add(street_name)
        else:
            street_types['None'].add(street_name)

def update_name(name, mapping_street):
    '''
    Correct the street name based on the type of errors.
    '''
    if name.split(' ')[0] != 'Jalan':
        if name.split(' ')[0] in mapping_street.keys():
            return name.replace(name.split(' ')[0],mapping_street[name.split(' ')[0]])
        elif len(name.split(',')) > 1 and name.split(',')[1][:4] == ' JL.':
            name_list = name.split(',')
            return name_list[1].replace(' JL.','Jalan') + ' ' + name_list[0]
        elif name.split(' ')[0][:3] == 'Jl.':
            return name.replace('Jl.','Jalan ')
        elif len(name.split(',')) > 1:
            if name.split(',')[0][:5] == 'Jalan':
                return name.split(',')[0][:-1]
            else:
                return 'Jalan '+ name.split(',')[0][:-1]
        elif name[-4:] == 'Road':
            return 'Jalan '+ name[:-5]
        else:
            return 'Jalan '+ name
    else:
        return name

def audit(osmfile):
    '''
    Audit and fix the street name.
    '''
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                    # update tag.attrib['v'] with the return from update_name()
                    tag.attrib['v'] = update_name(tag.attrib['v'],mapping_street)

    osm_file.close()
    return street_types

def test():
    '''
    Check if the code properly update the street name in a few examples.
    '''
    st_types = audit(OSMFILE)
    pprint.pprint(dict(st_types))
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping_street)
            if name == "Jln. Danau Poso":
                assert better_name == "Jalan Danau Poso"
            if name == "Jl.Gatot Subroto":
                assert better_name == "Jalan Gatot Subroto"
            if name == "Raya Kerobokan":
                assert better_name == "Jalan Raya Kerobokan"
            if name == "Jalan Bedugul Sidakarya, Denpasar - Bali":
                assert better_name == "Jalan Bedugul Sidakarya"
            if name == "tegalsari 37 pantai berawa ,Canggu":
                print(better_name)
                assert better_name == "Jalan tegalsari 37 pantai berawa"
            if name == "Hanoman Road":
                assert better_name == "Jalan Hanoman"
            if name == "Komplek Burung, JL. Elang":
                assert better_name == "Jalan Elang Komplek Burung"

if __name__ == '__main__':
    test()

{'JALAN': set(['JALAN PANTAI BALIAN']),
 'JL': set(['JL Banda Sari, Gang Poppies 2',
            'JL Dalung Permai',
            'JL Pantai Berawa',
            'JL Petitenget']),
 'JL.': set(['JL. Bakung Sari',
             'JL. By Pass Ngurah Rai',
             'JL. Dhyana Pura',
             'JL. Diponegoro,',
             'JL. Padma Legian',
             'JL. Padma Utara',
             'JL. Pantai Batu Bolong',
             'JL. Patih Jelantik',
             'JL. Poppies Lane I/27',
             'JL. Pucuk, Banjar Tangtu',
             'JL. RAYA BATU BOLONG 39 CANGGU, BALI',
             'JL. Singasari',
             'JL. Sunset Road',
             'JL. Tukad Taman Sari',
             'JL. Werkudara']),
 'JL._invert': set(['Komplek Burung, JL. Elang']),
 'JLN.': set(['JLN. Batu Mejan',
              'JLN. Dalem Lingsir',
              'JLN. LANYAHAN, BR NAGI,']),
 'Jalan.': set(['Jalan. Kampus Bukit Jimbaran']),
 'Jl': set(['Jl . Pantai Geger',
            'Jl Cendrawasih',
       

### Improving Phone Numbers

In [5]:
"""
This code fixes the phone number.
The function takes a string with phone number as an argument and returns the fixed phone number.
There are two types of inconsistencies:
1. Inconsistent country code
2. Incorrect phone number
Reference: this code was modified from 'Case study: OpenStreetMap Data[SQL] Quiz 10'
"""
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import pprint
import cerberus
import schema
from collections import Counter,defaultdict
OSMFILE = 'bali.osm'

codes = ['(+62)','(62)','(+62361)','62','p. +62','+62','0062','062','[+62]',
        '021',
        '(0361)','0361','0368',
        '080','081','082','083','085','087','089']

def is_phone_num(elem):
    '''
    Check if the element contains phone number.
    '''    
    if elem.attrib['k'] == "phone":
        if not elem.attrib['v'].startswith(tuple(codes)):
            print('Incorrect phone numbers:',elem.attrib['v'])
    
    return (elem.attrib['k'] == "phone" and 
            elem.attrib['v'].startswith(tuple(codes)))

def audit_phonenum_type(phonenum_types, phone_num):
    '''
    Classify the phone numbers based on types of errors.
    '''
    for code in codes:
        if phone_num.startswith(code):
            phonenum_types[code].add(phone_num)
            return

def update_phonenum(num):
    '''
    Fix the phone numbers based on the type of errors.
    '''
    code_with62 = ['62','0062','062']
    code_no62 = ['021','0361','0368',
                '080','081','082','083','085','087','089']
    if num.startswith('p. +62'):
        return num.lstrip('p. ')
    if num.startswith('('):
        for char in '()':
            num = num.replace(char,'')
    if num.startswith('['):
        for char in '[]':
            num = num.replace(char,'')
    if '(0)' in num:
        num = num.replace('(0)','')
    
    if num.startswith(tuple(code_with62)):
        for code in code_with62:
            if num.startswith(code):
                return '+62' + num.lstrip(code)
    elif num.startswith(tuple(code_no62)):
        for code in code_no62:
            if num.startswith(code):
                return '+62' + num.lstrip('0')
    else:
        return num

def audit(osmfile):
    '''
    Audit and fix the phone numbers.
    '''
    osm_file = open(osmfile, "r")
    phonenum_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_num(tag):
                    audit_phonenum_type(phonenum_types, tag.attrib['v'])
                    # update tag.attrib['v'] with the return from update_phonenum()
                    tag.attrib['v'] = update_phonenum(tag.attrib['v'])

    osm_file.close()
    return phonenum_types

def test():
    phonenum_types = audit(OSMFILE)
    pprint.pprint(dict(phonenum_types))
    for phonenum_type, ways in phonenum_types.iteritems():
        for num in ways:
            better_num = update_phonenum(num)

if __name__ == '__main__':
    test()

('Incorrect phone numbers:', '+85 738 481 121')
('Incorrect phone numbers:', '-8.676387, 115.155813')
('Incorrect phone numbers:', '1 500 310')
('Incorrect phone numbers:', '07245465')
('Incorrect phone numbers:', '+79684349570')
{'(+62)': set(['(+62) 361 911 6350']),
 '(+62361)': set(['(+62361)8972066']),
 '(0361)': set(['(0361) 703532',
                '(0361) 737082',
                '(0361) 752403',
                '(0361) 755277',
                '(0361) 756819',
                '(0361) 761484',
                '(0361) 7885811',
                '(0361) 971490',
                '(0361) 973314',
                '(0361) 974615',
                '(0361) 975054',
                '(0361) 977 688',
                '(0361) 977169']),
 '(62)': set(['(62) 361-479-2777']),
 '+62': set(['+62 (0) 361 977 922',
             '+62 (0) 851 0088 4968',
             '+62 (0)85857119842',
             '+62 (361) 730 840',
             '+62 (361) 848 3939',
             '+62 (361) 97 00 48',
         

### Parsing and shaping each element in the OSM file

In [6]:
"""
1. Use the iterparse to iteratively step through each top level element in XML 
2. Shaping each element into certain data structures
3. Write each data structure to the appropriate csv files
Reference: this code was modified from 'Case study: OpenStreetMap Data[SQL] Quiz 11'
Note: validation was run on 'sample.osm' and no response was obtained,
      indicating that the output structure matches the structure in schema.py
      The following code is run on 'bali.osm' without validation.
"""
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import pprint
import cerberus
import schema
from collections import Counter,defaultdict

OSM_PATH = "bali.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_fields=NODE_FIELDS, node_tags_fields=NODE_TAGS_FIELDS, 
                  way_fields=WAY_FIELDS, way_tags_fields=WAY_TAGS_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    # Handle secondary tags the same way for both node and way elements
    node_attribs, way_attribs = {}, {}
    tags, way_nodes = [], []
    
    #Parsing through each child element with node tag
    if element.tag == 'node':
        for node_attr in node_fields:
            node_attribs[node_attr] = element.attrib[node_attr]
        #Parsing through each lower element of the child elements with node tag
        subtags = element.iter('tag')
        if subtags:
            for tag in subtags:
                if not PROBLEMCHARS.search(tag.attrib['k']):
                    tag_dict = {}
                    tag_dict['id'] = element.attrib['id']
                    # use 'if is_street_name()' and 'if is_phone_num_'
                    # function to determine if the attribute matches
                    if is_street_name(tag):
                        tag.attrib['v']=update_name(tag.attrib['v'],mapping_street)
                    if is_phone_num(tag):
                        tag.attrib['v']=update_phonenum(tag.attrib['v'])
                    tag_dict['value'] = tag.attrib['v']
    
                    if LOWER_COLON.search(tag.attrib['k']):
                        split_key = (tag.attrib['k']).split(':')
                        tag_dict['type'] = split_key[0]
                        if len(split_key) == 2:
                            tag_dict['key'] = split_key[1]
                        elif len(split_key) > 2:
                            tag_dict['key'] = ':'.join(split_key[1:])
                    else:
                        tag_dict['type'] = default_tag_type
                        tag_dict['key'] = tag.attrib['k']
                    tags.append(tag_dict)
        else:
            tags.append([])
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        #Parsing through each child element with way tag
        for way_attr in way_fields:
            way_attribs[way_attr] = element.attrib[way_attr]
        #Parsing through each lower element of the child elements with way tag
        subtags_nd = element.iter('nd')
        if subtags_nd:
            counter = 0
            for subtag_nd in subtags_nd:
                way_node_dict = {}
                way_node_dict['id'] = element.attrib['id']
                way_node_dict['node_id'] = subtag_nd.attrib['ref']
                way_node_dict['position'] = counter
                counter += 1
                way_nodes.append(way_node_dict)
            
        subtags = element.iter('tag')
        if subtags:
            for tag in subtags:
                if not PROBLEMCHARS.search(tag.attrib['k']):
                    tag_dict = {}
                    tag_dict['id'] = element.attrib['id']
                    # use 'if is_street_name()' and 'if is_phone_num_'
                    # function to determine if the attribute matches
                    if is_street_name(tag):
                        tag.attrib['v']=update_name(tag.attrib['v'],mapping_street)
                    if is_phone_num(tag):
                        tag.attrib['v']=update_phonenum(tag.attrib['v'])
                        
                    tag_dict['value'] = tag.attrib['v']
                    #Checking if the 'k' attribute is 'address'
                    if tag.attrib['k'].strip() == "address":
                        print 'k attribute is address with the following value:'
                        print tag.attrib['v']
                    if LOWER_COLON.search(tag.attrib['k']):
                        split_key = (tag.attrib['k']).split(':')
                        tag_dict['type'] = split_key[0]
                        if len(split_key) == 2:
                            tag_dict['key'] = split_key[1]
                        elif len(split_key) > 2:
                            tag_dict['key'] = ':'.join(split_key[1:])
                    else:
                        tag_dict['type'] = default_tag_type
                        tag_dict['key'] = tag.attrib['k']
                tags.append(tag_dict)
        else:
            tags.append([])
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

('Incorrect phone numbers:', '+85 738 481 121')
('Incorrect phone numbers:', '-8.676387, 115.155813')
('Incorrect phone numbers:', '1 500 310')
('Incorrect phone numbers:', '07245465')
('Incorrect phone numbers:', '+79684349570')


### Import csv files into SQLite tables

In [39]:
import csv, sqlite3

conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.executescript(
'''
DROP TABLE IF EXISTS Nodes;
DROP TABLE IF EXISTS Nodes_tags;
DROP TABLE IF EXISTS Ways;
DROP TABLE IF EXISTS Ways_tags;
DROP TABLE IF EXISTS Ways_nodes;

CREATE TABLE Nodes (
    id  TEXT NOT NULL PRIMARY KEY UNIQUE,
    lat TEXT,
    lon TEXT,
    user TEXT,
    uid TEXT,
    version TEXT,
    changeset TEXT,
    timestamp TEXT
);

CREATE TABLE Nodes_tags (
    id  TEXT NOT NULL,
    key TEXT,
    value TEXT,
    type TEXT
);

CREATE TABLE Ways (
    id  TEXT NOT NULL PRIMARY KEY UNIQUE,
    user  TEXT,
    uid TEXT,
    version TEXT,
    changeset TEXT,
    timestamp TEXT
);

CREATE TABLE Ways_tags (
    id  INTEGER NOT NULL,
    key TEXT,
    value TEXT,
    type TEXT
);

CREATE TABLE Ways_nodes (
    id  TEXT NOT NULL,
    node_id TEXT NOT NULL,
    position TEXT
);
''')
# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes.csv','rb') as nodes_r:
    nodes_dr = csv.DictReader(nodes_r)
    nodes_to_db = [(row['id'],row['lat'],row['lon'],row['user'].decode('utf-8'),
              row['uid'],row['version'],row['changeset'],row['timestamp']) 
              for row in nodes_dr]

for idx in range(len(nodes_to_db)):
    cur.execute(
       '''
       INSERT INTO Nodes (id,lat,lon,user,uid,version,changeset,timestamp)
       VALUES (?,?,?,?,?,?,?,?);
       ''',nodes_to_db[idx])
conn.commit()

with open('nodes_tags.csv','rb') as nodes_tags_r:
    nodes_tags_dr = csv.DictReader(nodes_tags_r)
    nodes_tags_to_db = [(row['id'],row['key'],row['value'].decode('utf-8'),row['type']) 
                        for row in nodes_tags_dr]

for idx in range(len(nodes_tags_to_db)):
    cur.execute(
       '''
       INSERT INTO Nodes_tags (id,key,value,type)
       VALUES (?,?,?,?);
       ''',nodes_tags_to_db[idx])
conn.commit()

with open('ways.csv','rb') as ways_r:
    ways_dr = csv.DictReader(ways_r)
    ways_to_db = [(row['id'],row['user'].decode('utf-8'),row['uid'],
                   row['version'],row['changeset'],row['timestamp']) 
                   for row in ways_dr]

for idx in range(len(ways_to_db)):
    cur.execute(
       '''
       INSERT INTO Ways (id,user,uid,version,changeset,timestamp)
       VALUES (?,?,?,?,?,?);
       ''',ways_to_db[idx])
conn.commit()

with open('ways_tags.csv','rb') as ways_tags_r:
    ways_tags_dr = csv.DictReader(ways_tags_r)
    ways_tags_to_db = [(row['id'],row['key'],row['value'].decode('utf-8'),row['type']) 
                        for row in ways_tags_dr]

for idx in range(len(ways_tags_to_db)):
    cur.execute(
       '''
       INSERT INTO Ways_tags (id,key,value,type)
       VALUES (?,?,?,?);
       ''',ways_tags_to_db[idx])
conn.commit()

with open('ways_nodes.csv','rb') as ways_nodes_r:
    ways_nodes_dr = csv.DictReader(ways_nodes_r)
    ways_nodes_to_db = [(row['id'],row['node_id'],row['position']) 
                        for row in ways_nodes_dr]

for idx in range(len(ways_nodes_to_db)):
    cur.execute(
       '''
       INSERT INTO Ways_nodes (id,node_id,position)
       VALUES (?,?,?);
       ''',ways_nodes_to_db[idx])
conn.commit()


conn.close()

### Number of nodes

In [15]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('SELECT COUNT(*) FROM Nodes;')
print cur.fetchone()[0]
conn.close()

489536


### Number of ways

In [16]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('SELECT COUNT(*) FROM Ways;')
print cur.fetchone()[0]
conn.close()

79414


### Number of unique users

In [1]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT COUNT(DISTINCT(union_node_ways.uid)) 
FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) as union_node_ways;
''')
print cur.fetchone()[0]
conn.close()

1295


### Top 10 contributing users

In [9]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT union_node_ways.user, COUNT(*) as num
FROM (SELECT user FROM Nodes UNION ALL SELECT user FROM Ways) as union_node_ways
GROUP BY union_node_ways.user
ORDER BY num DESC
LIMIT 10;
''')
for entry in cur.fetchall():
    print entry[0].decode('utf-8'),entry[1]
conn.close()

HEN TAM 234502
everyone_sinks_starco 79438
Werner17a 29672
Bernhard Hiller 13752
mahau 13380
robgeb 12702
wwwFrank 11538
FresRe 8782
Eleonora Easter Henny 8478
osmjessian 8261


### Number of users who have only 1 post

In [21]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT COUNT(*)
FROM 
    (SELECT COUNT(*) as num, union_node_ways.user 
     FROM (SELECT user FROM Nodes UNION ALL SELECT user FROM Ways) as union_node_ways
     GROUP BY union_node_ways.user
     HAVING num = 1);
''')
print cur.fetchone()[0]
conn.close()

455


### Top 10 appearing amenities

In [23]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT value, COUNT(*) as num
FROM Nodes_tags
WHERE key = 'amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;
''')
for entry in cur.fetchall():
    print entry[0].decode('utf-8'),entry[1]
conn.close()

restaurant 1106
cafe 261
place_of_worship 243
atm 211
bank 167
fuel 111
bar 81
fast_food 79
pharmacy 68
parking 49


### Biggest religion

In [28]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT Nodes_tags.value, COUNT(*) as num
FROM Nodes_tags
    JOIN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value = 'place_of_worship') as sub
    ON Nodes_tags.id = sub.id
WHERE Nodes_tags.key = 'religion'
GROUP BY Nodes_tags.value
ORDER BY num DESC
LIMIT 1;
''')
for entry in cur.fetchall():
    print entry[0].decode('utf-8'),entry[1]
conn.close()

hindu 156


### Most popular cuisines

In [31]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT Nodes_tags.value, COUNT(*) as num
FROM Nodes_tags
    JOIN (SELECT DISTINCT(id) FROM Nodes_tags WHERE value = 'restaurant') as sub
    ON Nodes_tags.id = sub.id
WHERE Nodes_tags.key = 'cuisine'
GROUP BY Nodes_tags.value
ORDER BY num DESC
LIMIT 15;
''')
for entry in cur.fetchall():
    print entry[0].decode('utf-8'),entry[1]
conn.close()

regional 79
indonesian 47
asian 21
seafood 17
international 15
italian 14
chinese 13
japanese 13
vegetarian 11
pizza 7
indonesian;international 6
mexican 6
thai 5
indian 4
noodle 4


### Most popular shops

In [40]:
import sqlite3 
conn = sqlite3.connect('osmproject.sqlite')
cur = conn.cursor()
cur.execute('''
SELECT Nodes_tags.value, COUNT(*) as num
FROM Nodes_tags
    JOIN (SELECT DISTINCT(id) FROM Nodes_tags WHERE key = 'name') as sub
    ON Nodes_tags.id = sub.id
WHERE Nodes_tags.key = 'shop'
GROUP BY Nodes_tags.value
ORDER BY num DESC
LIMIT 10;
''')
for entry in cur.fetchall():
    print entry[0].decode('utf-8'),entry[1]
conn.close()

convenience 161
supermarket 90
clothes 58
bakery 45
massage 45
beauty 36
travel_agency 33
motorcycle 31
electronics 29
yes 27
